# Purpose

This notebook is to test implementation of Ensemble, Flipout and DUQ models.
- Question: Can the ensembles use dropout and dropconnect weights? Or are they to be trained without either.

In [1]:
# Model for ensemble:
# model = DeepEnsembleClassifier(lambda: build_standard_model(dropout_best_hps), num_estimators=10)     100 epochs
# DUQ: 200 epochs
# Flipout: 200 epochs

In [2]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [5]:
import sys, os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from models_bachelors import *
from file_functions import *
import tensorflow as tf
import keras_tuner as kt
from keras_uncertainty.models import StochasticClassifier
import time

In [6]:
'''
Load best hyperparams
'''

n_epochs= 200
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
subject_ids = [0, 1, 2, 3, 4, 5, 6, 7, 8]
'''
Load data
'''
dataset = load('all_subjects_runs_no_bandpass')
lockbox = load('lockbox')['data']
loaded_inputs = dataset['inputs']
loaded_targets = dataset['targets']

In [4]:
import time

class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [5]:
methods = {'flipout': [], 'duq': [], ''}
x_train_shape_0 = None


'''
Training Loop
'''
for method in methods:
    # This loop leaves one subject for testing (denoted by the number in the name of the weights file).
    # Then it combines all the subject trials such that shape is now (8 * 576, 22, 1125).
    # Then selects 10% of this as the validation set. Then it trains diff. model on each set of train subjects.
    for test_subject_id in subject_ids:
        train_ids = subject_ids[:]
        train_ids.remove(test_subject_id)       # Remove test subject id
        test_subj_lockbox = lockbox[test_subject_id]        # Get lockbox indexes (8, 57) for the test subject
        inputs = loaded_inputs[train_ids]           # Get train set inputs
        targets = loaded_targets[train_ids]         # Get train set targets
        inputs, targets = remove_lockbox(inputs, targets, test_subj_lockbox)    # Remove lockboxed set from train set
        X_train, X_val, Y_train, Y_val = train_test_split(inputs, targets,test_size=0.1) 
        if x_train_shape_0 == None:
            x_train_shape_0 = X_train.shape[0]
            hp = load_tuned_flipout(x_train_shape_0)

        model = build_flipout_model(hp, x_train_shape_0)
        start_time = time.time()
        history = model.fit(X_train, Y_train, epochs=n_epochs, validation_data=[X_val, Y_val],
                        callbacks=[early_stopping, time_callback], verbose=0)
        # model.save_weights(f'{directory}_test_subj_{test_subject_id}')
        

Reloading Tuner from flipout/tuning/flipout_2/tuner0.json


/home/pmanivannan/miniconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1260: UserWarning: `tf.keras.backend.random_binomial` is deprecated, and will be removed in a future version.Please use `tf.keras.backend.random_bernoulli` instead.
  return dispatch_target(*args, **kwargs)


In [7]:
methods = ['duq']
time_callback = TimeHistory()
times = None

hp = load_tuned_duq()

'''
Training Loop
'''
for method in methods:
    # This loop leaves one subject for testing (denoted by the number in the name of the weights file).
    # Then it combines all the subject trials such that shape is now (8 * 576, 22, 1125).
    # Then selects 10% of this as the validation set. Then it trains diff. model on each set of train subjects.
    for test_subject_id in subject_ids:
        train_ids = subject_ids[:]
        train_ids.remove(test_subject_id)       # Remove test subject id
        test_subj_lockbox = lockbox[test_subject_id]        # Get lockbox indexes (8, 57) for the test subject
        inputs = loaded_inputs[train_ids]           # Get train set inputs
        targets = loaded_targets[train_ids]         # Get train set targets
        inputs, targets = remove_lockbox(inputs, targets, test_subj_lockbox)    # Remove lockboxed set from train set
        X_train, X_val, Y_train, Y_val = train_test_split(inputs, targets,test_size=0.1) 
        if hp == None:         # Needed to load hyperparams for flipout
            # hp = load_tuned_flipout(X_train.shape[0])
            pass

        model = build_duq_model(hp)
        history = model.fit(X_train, Y_train, epochs=n_epochs, validation_data=[X_val, Y_val],
                        callbacks=[early_stopping, time_callback], verbose=0)
        times = time_callback.times
        break

print(f'{np.mean(times)} +/- {np.std(times)}')

Reloading Tuner from duq/tuning/duq_nunits_morethan_100/tuner0.json
Results summary
Results in duq/tuning/duq_nunits_morethan_100
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 017 summary
Hyperparameters:
dense_units: 100
dense_activation: relu
length_scale: 0.4
train_centroids: 1
Score: 0.3539585471153259

Trial 058 summary
Hyperparameters:
dense_units: 300
dense_activation: relu
length_scale: 0.5
train_centroids: 0
Score: 0.3559551537036896

Trial 074 summary
Hyperparameters:
dense_units: 400
dense_activation: relu
length_scale: 0.3
train_centroids: 0
Score: 0.35879483819007874

Trial 039 summary
Hyperparameters:
dense_units: 200
dense_activation: relu
length_scale: 0.5
train_centroids: 1
Score: 0.3617439568042755

Trial 038 summary
Hyperparameters:
dense_units: 200
dense_activation: relu
length_scale: 0.5
train_centroids: 0
Score: 0.3619697690010071

Trial 056 summary
Hyperparameters:
dense_units: 300
dense_activation: relu
length_scale: 0.4
train_centroi

In [8]:
methods = ['standard_dropout']
time_callback = TimeHistory()
times = None

dropout_best_hps, dropconnect_best_hps = load_tuned_models()

'''
Training Loop standard
'''
# Training loop for MCDropout and MCDropconnect methods
for method in methods:
    for test_subject_id in subject_ids:
        train_ids = subject_ids[:]
        train_ids.remove(test_subject_id)       # Remove test subject id
        test_subj_lockbox = lockbox[test_subject_id]        # Get lockbox indexes (8, 57) for the test subject
        inputs = loaded_inputs[train_ids]           # Get train set inputs
        targets = loaded_targets[train_ids]         # Get train set targets
        inputs, targets = remove_lockbox(inputs, targets, test_subj_lockbox)    # Remove lockboxed set from train set
        X_train, X_val, Y_train, Y_val = train_test_split(inputs, targets,test_size=0.1)
        
        model = build_standard_model(dropout_best_hps)
        history = model.fit(X_train, Y_train, epochs=n_epochs, validation_data=[X_val, Y_val],
                        callbacks=[early_stopping, time_callback], verbose=0)
        times = time_callback.times
        break

print(f'{np.mean(times)} +/- {np.std(times)}')  
                

Reloading Tuner from mcdropout/tuning/mcdropout/tuner0.json
Reloading Tuner from mcdropconnect/tuning/mcdropconnect/tuner0.json
3.146103620529175 +/- 0.46615941000979116


In [9]:
from keras_uncertainty.models import DeepEnsembleClassifier

methods = ['ensemble_dropout']
time_callback = TimeHistory()
times = None

dropout_best_hps, dropconnect_best_hps = load_tuned_models()

'''
Training Loop standard
'''
# Training loop for MCDropout and MCDropconnect methods
for method in methods:
    for test_subject_id in subject_ids:
        train_ids = subject_ids[:]
        train_ids.remove(test_subject_id)       # Remove test subject id
        test_subj_lockbox = lockbox[test_subject_id]        # Get lockbox indexes (8, 57) for the test subject
        inputs = loaded_inputs[train_ids]           # Get train set inputs
        targets = loaded_targets[train_ids]         # Get train set targets
        inputs, targets = remove_lockbox(inputs, targets, test_subj_lockbox)    # Remove lockboxed set from train set
        X_train, X_val, Y_train, Y_val = train_test_split(inputs, targets,test_size=0.1)
        model = DeepEnsembleClassifier(lambda: build_standard_model(dropout_best_hps), num_estimators=10) 
        history = model.fit(X_train, Y_train, epochs=n_epochs, validation_data=[X_val, Y_val],
                        callbacks=[early_stopping, time_callback], verbose=0)
        times = time_callback.times
        break

print(f'{np.mean(times)} +/- {np.std(times)}')   
                

Reloading Tuner from mcdropout/tuning/mcdropout/tuner0.json
Reloading Tuner from mcdropconnect/tuning/mcdropconnect/tuner0.json


# Predicting

But only to record the inference time. Actual predicting is done by modification and execution of the file 'predict_new_uq.py'

In [ ]:
NUM = 1
# For each iteration, store results dict into a 
for iteration in range(0, NUM):
    # For each method, get preds and labels for each test subject
    # and their corresponding lockbox set.
    times =     {'standard': [],
                'standard_dropconnect': [],
                'duq': [],
                'flipout': [],
                'ensemble_dropout': [],
                'mcdropout': [],
                'mcdropconnect': []
                }

    for method, values in predictions.items():
        print(f'{method}')
        if method == 'standard':
            wts_directory = f'mcdropout/weights'
        elif method == 'standard_dropconnect':
            wts_directory = f'mcdropconnect/weights'
        else:
            wts_directory = f'{method}/weights'
        # Iterate through test subjects
        for test_subject_id in range(0, 9):
            print(f'test subject {test_subject_id}')
            train_subj_ids = [x for x in subject_ids if x != test_subject_id]
            X_test = loaded_inputs[test_subject_id]
            Y_true = loaded_targets[test_subject_id]
            # Train set is sent in because lockbox is returned from the train set not the whole dataset.
            # This is because lockbox shape: (9, 8, 57) and inputs shape: (9, 576, 22, 1125)
            # Axis 0 are test_subj_ids and axis 1 are the train_subject_ids.
            # The function assumes that shape[0] of lockbox[test_subj_id] and shape[0] of
            # inputs is the same: 8.
            X_lock, Y_lock = get_lockbox_data(loaded_inputs[train_subj_ids], loaded_targets[train_subj_ids], lockbox[test_subject_id])
            wts_path = checkpoint_path = f'{wts_directory}/test_subj_{test_subject_id}'
            if method == 'mcdropout':
                model = build_dropout_model(dropout_best_hps)
            elif method == 'mcdropconnect':
                model = build_dropconnect_model(dropconnect_best_hps)
            elif method == 'standard_dropconnect':
                model = build_standard_model_dropconnect(dropconnect_best_hps)
            elif method == 'duq':
                model = build_duq_model(hp)
            elif method == 'flipout':
                model = build_flipout_model(hp, X_test)
            else:
                model = build_standard_model(dropout_best_hps)
            
            model.load_weights(wts_path).expect_partial()
            # Get Y_preds for test subject
            if method in ['mcdropconnect', 'mcdropout', 'flipout']:
                model = StochasticClassifier(model)
                Y_preds = model.predict_samples(X_test, num_samples=50)
                # Get lockboxed Y_preds for test subject
                lockbox_Y_preds = model.predict_samples(X_lock, num_samples=50)
            else:
                Y_preds = model.predict(X_test)
                # Get lockboxed Y_preds for test subject
                lockbox_Y_preds = model.predict(X_lock)